In [1]:
# Importing necessary models
import warnings
warnings.filterwarnings('ignore')

import smtplib
import pandas as pd
import numpy as np
import datetime as dt
pd.options.display.float_format = '{:,.4f}'.format
from scipy.stats import norm as norm
import os
from sqlalchemy import *
from sqlalchemy import create_engine
import time
%matplotlib inline
dbs_dir = 'C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Data\\DBs'

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\DataCollection')

from yahoo_query import *

vix_options_dir = 'D:\\Options Data\\VIX'
spx_options_dir = 'D:\\Options Data\\SPX'
spx_intraday_dir = 'D:\\Options Data\\SPX Intraday'
cleaned_dir = 'D:\\Options Data\\Aggregated DBs'

def delta_0dte(options_df):
    d1 = (np.log(options_df['Underlying_Price'] / 
                 options_df['Strike']) + 
          (options_df['IV']**2 / 2.0) * np.sqrt(options_df['TimeRemaining'])) / (options_df['IV'] * np.sqrt(options_df['TimeRemaining']))
    if options_df.Type[0] == 'call':
        return norm.cdf(d1)
    elif options_df.Type[0] == 'put':
        return 1 - norm.cdf(d1)
    
def gamma_0dte(options_df):
    d1 = (np.log(options_df['Underlying_Price'] / 
                 options_df['Strike']) + 
          (options_df['IV']**2 / 2.0) * np.sqrt(options_df['TimeRemaining'])) / (options_df['IV'] * np.sqrt(options_df['TimeRemaining']))

    return norm.pdf(d1)/(options_df['Underlying_Price']*options_df['IV']*np.sqrt(options_df['TimeRemaining']))

def spx_intraday_prices(read_date):

    os.chdir(spx_intraday_dir)
    filename = 'spx_{}.csv'.format(read_date.strftime("%Y-%m-%d"))
    if filename in os.listdir('.'):
        spx_intraday = pd.read_csv(filename, index_col = 0)
        spx_intraday.index = pd.to_datetime(spx_intraday.index)
        
        try:
            spx_intraday = spx_intraday[spx_intraday.index.time >= dt.time(9,30)]
        except:
            None
        
        os.chdir(cleaned_dir)
        curr_date = spx_intraday.index[0]
        
        latest_month = curr_date.month
        
        db_name = 'sqlite:///spx_intraday_{0}-{1}.db'.format(curr_date.year,latest_month)

        table_name = 'day_{}'.format(curr_date.strftime("%Y-%m-%d"))

        spx_price_engine = create_engine(db_name, echo=False)

        spx_intraday.to_sql(table_name, con=spx_price_engine, if_exists='append')
    
        return spx_intraday
    else:
        print('No Data')
        return 'None'

def write_daily_data(spx_options_filenames, vix_options_filenames):
    start_time = time.time()
    
    spx_options_cols = ['Strike', 'Expiry', 'Type', 'Symbol', 'Last', 'Bid', 'Ask', 'Vol',
                        'Open_Int', 'IV', 'Root', 'Underlying_Price', 'Quote_Time']
    vix_options_cols = ['Quote_Time','Underlying_Price']

    spx_daily_options = []
    os.chdir(spx_options_dir)

    for file in spx_options_filenames:
        spx_daily_options.append(pd.read_csv(file,index_col = 0)[spx_options_cols])
    spx_daily_options = pd.concat(spx_daily_options, axis = 0).drop_duplicates().sort_values('Quote_Time')

    vix_daily_options = []
    os.chdir(vix_options_dir)
    for file in vix_options_filenames:
        vix_daily_options.append(pd.read_csv(file, index_col = 0)[vix_options_cols])
    vix_daily_options = pd.concat(vix_daily_options, axis = 0).drop_duplicates().sort_values('Quote_Time')

    spx_daily_options.Quote_Time = pd.to_datetime(spx_daily_options.Quote_Time).map(lambda x: x.replace(second=0))
    spx_daily_options.Expiry = pd.to_datetime(spx_daily_options.Expiry)

    vix_daily_options.Quote_Time = pd.to_datetime(vix_daily_options.Quote_Time).map(lambda x: x.replace(second=0))
    vix_daily_options.columns = ['Quote_Time', 'VIX']

    spx_time_adj = spx_daily_options['Quote_Time'].iloc[0].replace(year=2018,month=1,day=1,minute=0) - dt.datetime(2018,1,1,9)
    vix_time_adj = vix_daily_options['Quote_Time'].iloc[0].replace(year=2018,month=1,day=1,minute=0) - dt.datetime(2018,1,1,9)

    spx_daily_options.Quote_Time = spx_daily_options.Quote_Time - spx_time_adj
    vix_daily_options.Quote_Time = vix_daily_options.Quote_Time - vix_time_adj

    spx_daily_options['DTE'] = (spx_daily_options.Expiry - spx_daily_options.Quote_Time).dt.days + 1
    spx_daily_options.Expiry = spx_daily_options.Expiry + dt.timedelta(seconds = 60*60*16 + 1)
    spx_daily_options = spx_daily_options[spx_daily_options['DTE'] <= 1].reset_index(drop = True)
    spx_daily_options['TradeDate'] = spx_daily_options.Quote_Time.dt.date
    keep_contracts = spx_daily_options[['TradeDate','Symbol']].groupby('Symbol').count().sort_values('TradeDate')
    spx_daily_options = spx_daily_options[spx_daily_options.Symbol.isin(keep_contracts[keep_contracts.TradeDate >= 79].index)].reset_index(drop = True)

    options_df = pd.merge_asof(spx_daily_options.sort_values('Quote_Time'), 
                               vix_daily_options.sort_values('Quote_Time'),
                               on='Quote_Time', tolerance=pd.Timedelta('5m'), allow_exact_matches=True, direction='nearest')
    if len(options_df) != 0:
        options_df['TimeRemaining'] = (options_df['Expiry'] - options_df['Quote_Time']).dt.seconds/(60*60*24*365.25)
        
        calls = options_df[(options_df.Type == 'call')].reset_index(drop = True)
        puts = options_df[(options_df.Type == 'put')].reset_index(drop = True)
        
        
        calls['Delta'] = delta_0dte(calls)
        calls['Gamma'] = gamma_0dte(calls)
        
        puts['Delta'] = delta_0dte(puts)
        puts['Gamma'] = gamma_0dte(puts)
        
        options_df = pd.concat([calls,puts], axis = 0).reset_index(drop = True)

        os.chdir(cleaned_dir)
        curr_date = options_df.loc[0,'TradeDate']
        
        latest_month = curr_date.month
        db_name = 'sqlite:///options_{0}-{1}.db'.format(curr_date.year,latest_month)

        table_name = 'day_{}'.format(curr_date.strftime("%Y-%m-%d"))

        options_engine = create_engine(db_name, echo=False)

        options_df.to_sql(table_name, con=options_engine, if_exists='append')
    print("Completed in {} seconds".format(time.time() - start_time))
    return vix_daily_options.set_index('Quote_Time')



In [ ]:
curr_day = dt.datetime(2019,1,22)

while curr_day < dt.datetime.today():
    
    curr_date_string = curr_day.strftime("%Y-%m-%d")
    
    os.chdir(spx_options_dir)
    spx_options_filenames = list(filter(lambda x: curr_date_string in x, os.listdir('.')))
    
    os.chdir(vix_options_dir)
    vix_options_filenames = list(filter(lambda x: curr_date_string in x, os.listdir('.')))
    if spx_options_filenames != []:
        intraday_vix = write_daily_data(spx_options_filenames, vix_options_filenames)
        intraday_spx = spx_intraday_prices(curr_day)
        
        if type(intraday_spx) != str:
            latest_indices = intraday_vix.join(intraday_spx)
            latest_indices = latest_indices[latest_indices.index.time >= dt.time(9,30)]
            
            if len(latest_indices) != 0:
                os.chdir(cleaned_dir)
                curr_date = latest_indices.index[0]
                latest_month = curr_date.month
                db_name = 'sqlite:///indices_intraday_{0}-{1}.db'.format(curr_date.year,latest_month)

                table_name = 'day_{}'.format(curr_date.strftime("%Y-%m-%d"))

                vix_intraday_engine = create_engine(db_name, echo=False)

                latest_indices.to_sql(table_name, con=vix_intraday_engine, if_exists='append')
            
    curr_day = curr_day + dt.timedelta(days = 1)
    

In [3]:
curr_day = dt.datetime(2019,2,11)

os.chdir(spx_options_dir)


curr_date_string = curr_day.strftime("%Y-%m-%d")
spx_options_filenames = list(filter(lambda x: curr_date_string in x, os.listdir('.')))

In [24]:
test_spx = pd.read_csv(spx_options_filenames[5], index_col = 0)
test_spx.Expiry = pd.to_datetime(test_spx.Expiry)
test_spx.Quote_Time = pd.to_datetime(test_spx.Quote_Time)
test_spx = test_spx[(test_spx.Expiry - test_spx.Quote_Time).dt.days <= 90]
test_spx['Mid'] = (test_spx.Bid + test_spx.Ask)/2

test_calls = test_spx[(test_spx.Type == 'call') & 
                      (abs(test_spx.Strike - test_spx.Underlying_Price) <= 50)].reset_index(drop = True)
test_puts = test_spx[(test_spx.Type == 'put') & 
                      (abs(test_spx.Strike - test_spx.Underlying_Price) <= 50)].reset_index(drop = True)

In [63]:
test_calls['t'] = np.nan

for idx, row in test_calls.iterrows():
    
    t_array = np.linspace(0,1,10000)
    
    d1 = (np.log(row.Underlying_Price/row.Strike) + ((row.IV**2)/2)*t_array)/(row.IV*np.sqrt(t_array))
    d2 = d1 - row.IV*np.sqrt(t_array)
    
    c = row.Underlying_Price*norm.cdf(d1) - row.Strike*norm.cdf(d2)
    test_calls.loc[idx,'t'] = t_array[np.argmin(abs(c - row.Mid))]